In [ ]:
!pip install torch torchvision torchaudio --quiet

In [ ]:
import os
import pandas as pd
import numpy as np

import kagglehub

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from torchvision import models, transforms
from PIL import Image
from tqdm.auto import tqdm


In [ ]:
path_ = kagglehub.dataset_download(
    "asefjamilajwad/car-crash-dataset-ccd",
    force_download=False
)

print(path_)

Using Colab cache for faster access to the 'car-crash-dataset-ccd' dataset.
/kaggle/input/car-crash-dataset-ccd


In [ ]:
BASE_DIR = path_
CRASH_DIR = os.path.join(BASE_DIR, "CrashBest")
CSV_PATH = os.path.join(BASE_DIR, "Crash_Table.csv")

df = pd.read_csv(CSV_PATH)


In [ ]:
frame_cols = [f"frame_{i}" for i in range(1, 51)]


### Build frame-level samples

In [ ]:
samples = []

for _, row in df.iterrows():
    vid = int(row["vidname"])
    prefix = f"C_{vid:06d}"

    for i in range(1, 51):
        img_path = f"{CRASH_DIR}/{prefix}_{i:02d}.jpg"
        label = int(row[f"frame_{i}"])
        samples.append((img_path, label))


In [ ]:
len(samples), samples[:3]


(75000,
 [('/kaggle/input/car-crash-dataset-ccd/CrashBest/C_000001_01.jpg', 0),
  ('/kaggle/input/car-crash-dataset-ccd/CrashBest/C_000001_02.jpg', 0),
  ('/kaggle/input/car-crash-dataset-ccd/CrashBest/C_000001_03.jpg', 0)])

## Video-level train / validation split (leakage but were doing frame lavel)

In [ ]:
train_samples, val_samples = train_test_split(
    samples,
    test_size=0.2,
    random_state=42,
    stratify=[s[1] for s in samples]
)

### Custom PyTorch Dataset


In [ ]:
class CrashFrameDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)


In [ ]:
### imageneeeeet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


In [ ]:
train_ds = CrashFrameDataset(train_samples, transform)
val_ds   = CrashFrameDataset(val_samples, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)


### MobileNetV2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(model.last_channel, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 106MB/s] 


### traiining setup

In [ ]:
def run_epoch(model, loader, train=True):
    model.train() if train else model.eval()
    total_loss = 0.0
    correct = 0

    pbar = tqdm(loader, leave=False, desc="train" if train else "val")

    with torch.set_grad_enabled(train):
        for x, y in pbar:
            x = x.to(device)
            y = y.to(device)

            if train:
                optimizer.zero_grad()

            logits = model(x).squeeze(1)
            loss = criterion(logits, y)

            if train:
                loss.backward()
                optimizer.step()

            preds = (torch.sigmoid(logits) > 0.5).float()
            correct += (preds == y).sum().item()
            total_loss += loss.item()

            pbar.set_postfix(loss=loss.item())

    acc = correct / len(loader.dataset)
    return total_loss / len(loader), acc


In [ ]:
epochs = 5
for e in range(5):
    print(f"\nEpoch {e+1}/5")

    train_loss, train_acc = run_epoch(model, train_loader, train=True)
    val_loss, val_acc = run_epoch(model, val_loader, train=False)

    print(
        f"train acc: {train_acc:.3f} | "
        f"val acc: {val_acc:.3f}"
    )



Epoch 1/5


train:   0%|          | 0/1875 [00:00<?, ?it/s]

val:   0%|          | 0/469 [00:00<?, ?it/s]

train acc: 0.851 | val acc: 0.910

Epoch 2/5


train:   0%|          | 0/1875 [00:00<?, ?it/s]

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3, 224, 224).to(device)
scripted_model = torch.jit.trace(model, dummy_input)

MODEL_PATH = "/content/accident_classifier.ts"
scripted_model.save(MODEL_PATH)

MODEL_PATH


In [ ]:
from google.colab import files
files.download(MODEL_PATH)